## Load and clean the data

In [4]:
# In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from
# https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
# In this dataset our desired target for classification task will be converted variable - has the client signed up to the platform or not.

# Data preparation
# Check if the missing values are presented in the features.
# If there are missing values:
# For caterogiral features, replace them with 'NA'
# For numerical features, replace with with 0.0
# Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution. 
# Use train_test_split function for that with random_state=1

# run once to refresh the dataset csv file -
# !wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv

In [5]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

df = pd.read_csv('course_lead_scoring.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [6]:
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']
numerical 

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [7]:
categorical = ['lead_source', 'industry', 'employment_status', 'location']
categorical

['lead_source', 'industry', 'employment_status', 'location']

In [8]:
# how many missing values in our dataframe per column
df.isna().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [9]:
# If there are missing values:
# -- For categorical features, replace them with 'NA'

df[categorical] = df[categorical].fillna('NA') # to fill NaN in-place
for col in df[categorical]:
    print(f"{col}:")
    print(df[col].unique())
    print("-" * 40)

lead_source:
['paid_ads' 'social_media' 'events' 'referral' 'organic_search' 'NA']
----------------------------------------
industry:
['NA' 'retail' 'healthcare' 'education' 'manufacturing' 'technology'
 'other' 'finance']
----------------------------------------
employment_status:
['unemployed' 'employed' 'NA' 'self_employed' 'student']
----------------------------------------
location:
['south_america' 'australia' 'europe' 'africa' 'middle_east' 'NA'
 'north_america' 'asia']
----------------------------------------


In [10]:
# If there are missing values:
# -- For numerical features, replace with with 0.0

df[numerical] = df[numerical].fillna(0.0) # to fill NaN in-place
# CHECK - how many missing values in our dataframe per column NOW:
df.isna().sum() # all good!

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64